In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import subprocess
from pathlib import Path
from glob import glob
import re
import os
from os.path import join as pjoin

In [5]:
dry_run = False

RAW_DIR = '/Users/richard/Desktop/Raw/'
OUT_DIR = '/Users/richard/Desktop/Motorized-Dataset/'
TRAJS_DIR = OUT_DIR
trajs_per_vid = 5

renderer_path = '../PanoRenderer/build/main'
vids_paths = sorted(glob(pjoin(RAW_DIR, '*.mp4')))
trajs_paths = sorted(glob(pjoin(TRAJS_DIR, 't*.npy')))
yc_paths = [i[:-4] + '-yc.npy' for i in vids_paths]

In [6]:
render_traj_path = 'tmp-render-traj.npy'


traj_idx = 0
for vid_path, yc_path in zip(vids_paths, yc_paths):

    #a, b, v_id, _ = parse_vid_name(vid_path)
    #tqdm.write(f'Processing video {a}-{b}-v{v_id}')

    # Sometimes OpenCV fails to read the last few frames of the video.
    # So we subtract 30 frames to be safe
    cap = cv2.VideoCapture(vid_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 30
    cap.release()

    vid_stem = Path(vid_path).stem
    print(f'Processing video {vid_stem}')

    for _ in range(trajs_per_vid):

        traj_stem = Path(trajs_paths[traj_idx]).stem
        print(f'\tUsing trajectory {traj_stem}...')

        # Load desired trajectory & apply yaw correction
        traj = np.load(trajs_paths[traj_idx])

        # Random start frame
        try:
            start_frame = np.random.randint(0, total_frames - len(traj))
        except:
            continue

        yc = np.load(yc_path)
        for i, e in enumerate(traj, start=start_frame):
            e[0] += yc[i]
        np.save(render_traj_path, traj)

        output_vid_path = pjoin(OUT_DIR, f'{traj_stem}-{vid_stem}.mp4')

        render_cmd = [
            renderer_path, vid_path, render_traj_path,
            output_vid_path, str(start_frame)]

        if dry_run:
            print('\t\t' + ' '.join(render_cmd))
        else:
            result = subprocess.run(render_cmd)
            assert result.returncode == 0

        traj_idx += 1

try:
    os.remove(render_traj_path)
except Exception as e:
    print(e)

Processing video v00
	Using trajectory t000...
	Using trajectory t001...
	Using trajectory t002...
	Using trajectory t003...
	Using trajectory t004...
Processing video v01
	Using trajectory t005...
	Using trajectory t006...
	Using trajectory t007...
	Using trajectory t008...
	Using trajectory t009...
Processing video v02
	Using trajectory t010...
	Using trajectory t011...
	Using trajectory t012...
	Using trajectory t013...
	Using trajectory t014...
Processing video v03
	Using trajectory t015...
	Using trajectory t016...
	Using trajectory t017...
	Using trajectory t018...
	Using trajectory t019...
Processing video v04
	Using trajectory t020...
	Using trajectory t021...
	Using trajectory t022...
	Using trajectory t023...
	Using trajectory t024...
Processing video v05
	Using trajectory t025...
	Using trajectory t026...
	Using trajectory t027...
	Using trajectory t028...
	Using trajectory t029...
